In [1]:
import pandas as pd
from pathlib import Path

pd.options.mode.copy_on_write = True

Объединение наземных данных за 2021-2023

In [2]:
logs = list(Path(".").rglob("*.log"))
df_lst_21_23 = pd.concat([pd.read_csv(log, sep="|") for log in logs]).drop(columns=["TF", "SurfTermo", "SurfTermoEqu", "ConductSBE", "ScSBE", "ScSBEequ", "WaterIR", "AirIR", "Speed", "Cours"])
df_lst_21_23 = df_lst_21_23.rename(columns={"DateTime" : "Time", "TermoSBE": "T"})
df_lst_21_23["T"] = df_lst_21_23['T'].astype(str).str.replace(',', '.')
df_lst_21_23["Lat"] = df_lst_21_23['Lat'].astype(str).str.replace(',', '.')
df_lst_21_23["Lon"] = df_lst_21_23['Lon'].astype(str).str.replace(',', '.')
df_lst_21_23 = df_lst_21_23[df_lst_21_23["Lat"].astype(float) != 0] # какие-то лишние измерения
df_lst_21_23.to_csv("Наземеные_измерения/ready/Temp21-23.csv", index=False)

KeyboardInterrupt: 

Объединение наземных данных за 2024

In [9]:
df_june = pd.read_csv('Наземеные_измерения/ready/TempJune.csv')
df_july = pd.read_csv('Наземеные_измерения/ready/TempJuly.csv')
df_sept = pd.read_csv('Наземеные_измерения/ready/TempSept.csv')
df_lst_24 = pd.concat([df_june, df_july, df_sept]).drop(columns="Speed")
old_format = "%d.%m.%Y %H:%M"
new_format = "%d.%m.%Y %H:%M:%S"
df_lst_24["Time"] = df_lst_24["Time"].apply(lambda dt: pd.to_datetime(dt, format=old_format))
df_lst_24["Time"] = df_lst_24["Time"].dt.strftime(new_format)
df_lst_24["T"] = df_lst_24['T'].astype(str).str.replace(',', '.')
df_lst_24["Lat"] = df_lst_24['Lat'].astype(str).str.replace(',', '.')
df_lst_24["Lon"] = df_lst_24['Lon'].astype(str).str.replace(',', '.')
df_lst_24.to_csv('Наземеные_измерения/ready/Temp24.csv', index=False)

Объединение данных за 2021-2024

In [ ]:
df_lst = pd.concat([df_lst_21_23, df_lst_24])
df_lst.to_csv('Наземеные_измерения/ready/Temp21-24.csv', index=False)

Сокращение данных, чтобы они шли не чаще чем с определённой частотой

In [3]:
def parse_dateTime(df: pd.DataFrame) -> pd.DataFrame:
   format = "%d.%m.%Y  %H:%M:%S"
   return df["Time"].apply(lambda dt: pd.to_datetime(dt, format=format))

df_lst = pd.read_csv('Наземеные_измерения/ready/Temp21-24.csv')
df_lst["Time"] = parse_dateTime(df_lst)
df_lst = df_lst.sort_values(by=['Time'])
df_lst

,Time,T,Lat,Lon
731173,2021-05-25 18:42:01,2.56830,51.896156,105.098007
731174,2021-05-25 18:42:06,2.58880,51.896130,105.098320
731175,2021-05-25 18:42:14,2.61300,51.896072,105.098824
731176,2021-05-25 18:42:19,2.60840,51.896034,105.099136
731177,2021-05-25 18:42:27,2.62660,51.895969,105.099632
...,...,...,...,...
1733140,2024-10-03 14:37:00,9.79160,51.847950,104.870730
1733141,2024-10-03 14:38:00,9.79175,51.847950,104.870730
1733142,2024-10-03 14:39:00,9.79290,51.847950,104.870730
1733143,2024-10-03 14:40:00,9.79430,51.847950,104.870730


In [5]:
def shorten_data(df: pd.DataFrame, limit_frequency_minutes: int) -> pd.DataFrame:
   margin = pd.Timedelta(limit_frequency_minutes, 'minutes')
   i = 0
   while i<df.shape[0]:
      time=df["Time"].iloc[i]
      start_dt = time - margin
      end_dt = time + margin
      df = df[(df["Time"] == time) | (df["Time"] <= start_dt) | (end_dt <= df["Time"])]
      i = i + 1;
   return df

df_lst_15 = shorten_data(df_lst, 15)
df_lst_1 = shorten_data(df_lst, 1)

In [6]:
df_lst_15.to_csv('Наземеные_измерения/ready/Temp21-24_cut15.csv', index=False)
df_lst_1.to_csv('Наземеные_измерения/ready/Temp21-24_cut1.csv', index=False)

# Фильтрация точек около берега

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')
baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')

baikal_geometry = baikal_shape.geometry()
baikal_polylines = baikal_geometry.coordinates()
baikal_combined_points = []
for point_list in baikal_polylines.getInfo():
   for point in point_list:
      baikal_combined_points.append(ee.Geometry.Point(point[0], point[1]))
baikal_combined_points
baikal_points =  ee.Geometry.MultiPoint(baikal_combined_points)

## Modis

In [2]:
result_15_read = pd.read_csv('Данные_валидации/modis_validation_ext_15_distance_filtered.csv')
result_15_read

,Time,T,Lat,Lon,Image Time,Image T,MODIS,Day/Night,distance_to_shore_m,T diff
0,2021-05-27 11:02:10+08:00,2.22800,52.465443,106.586891,2021-05-27 11:06:00+08:00,2.35,Modis11_Terra,Day,10263.662946,0.12200
1,2021-05-28 11:37:54+08:00,1.42000,52.052254,105.978180,2021-05-28 11:48:00+08:00,1.69,Modis11_Terra,Day,13368.793886,0.27000
2,2021-06-03 10:58:21+08:00,2.44340,51.606743,104.731491,2021-06-03 11:12:00+08:00,3.05,Modis11_Terra,Day,14584.820130,0.60660
3,2021-06-05 10:50:54+08:00,10.08810,52.292828,106.244370,2021-06-05 11:00:00+08:00,10.57,Modis11_Terra,Day,1620.903932,0.48190
4,2021-06-12 11:04:02+08:00,2.01110,53.814648,109.120308,2021-06-12 11:06:00+08:00,2.29,Modis11_Terra,Day,3250.812287,0.27890
...,...,...,...,...,...,...,...,...,...,...
720,2024-09-20 04:11:00+08:00,13.05815,51.608270,105.100390,2024-09-19 20:12:00+00:00,11.31,Modis21_Aqua,Night,5086.948500,-1.74815
721,2024-09-21 03:25:00+08:00,10.60505,51.876760,105.205520,2024-09-20 19:24:00+00:00,7.93,Modis21_Aqua,Night,3760.460237,-2.67505
722,2024-09-22 03:58:00+08:00,11.99560,52.100490,106.120810,2024-09-21 20:12:00+00:00,11.61,Modis21_Aqua,Night,6311.388764,-0.38560
723,2024-09-23 05:01:00+08:00,11.86775,52.575820,106.606930,2024-09-22 20:48:00+00:00,10.59,Modis21_Aqua,Night,13472.346742,-1.27775


In [ ]:
point_distance = result_15_read[['Lon','Lat']].apply(lambda x: (ee.Geometry.Point(*x)).distance(baikal_points).getInfo(), axis=1)
result_15_with_distance = result_15_read.assign(distance_to_shore_m=pd.Series(point_distance).values)

In [ ]:
result_15_with_distance.to_csv('Данные_валидации/modis_validation_ext_15_distance.csv', index=False)

Добавление колонки с разностью

In [4]:
result_15_read = pd.read_csv('Данные_валидации/modis_validation_ext_15_distance.csv')
result_15_read

,Time,T,Lat,Lon,Image Time,Image T,MODIS,Day/Night,distance_to_shore_m
0,2021-05-27 11:02:10+08:00,2.22800,52.465443,106.586891,2021-05-27 11:06:00+08:00,2.35,Modis11_Terra,Day,10263.662946
1,2021-05-28 11:37:54+08:00,1.42000,52.052254,105.978180,2021-05-28 11:48:00+08:00,1.69,Modis11_Terra,Day,13368.793886
2,2021-06-03 10:58:21+08:00,2.44340,51.606743,104.731491,2021-06-03 11:12:00+08:00,3.05,Modis11_Terra,Day,14584.820130
3,2021-06-04 11:49:31+08:00,2.57930,51.626816,105.359306,2021-06-04 11:54:00+08:00,4.11,Modis11_Terra,Day,3211.487032
4,2021-06-05 10:50:54+08:00,10.08810,52.292828,106.244370,2021-06-05 11:00:00+08:00,10.57,Modis11_Terra,Day,1620.903932
...,...,...,...,...,...,...,...,...,...
807,2024-09-21 03:25:00+08:00,10.60505,51.876760,105.205520,2024-09-20 19:24:00+00:00,7.93,Modis21_Aqua,Night,3760.460237
808,2024-09-22 03:58:00+08:00,11.99560,52.100490,106.120810,2024-09-21 20:12:00+00:00,11.61,Modis21_Aqua,Night,6311.388764
809,2024-09-23 05:01:00+08:00,11.86775,52.575820,106.606930,2024-09-22 20:48:00+00:00,10.59,Modis21_Aqua,Night,13472.346742
810,2024-09-25 03:36:00+08:00,12.23610,53.521020,108.145930,2024-09-24 19:48:00+00:00,9.59,Modis21_Aqua,Night,24556.604522


In [5]:
result_15_read["T diff"] = result_15_read["Image T"] - result_15_read["T"]
result_15_read.to_csv('Данные_валидации/modis_validation_ext_15_distance.csv', index=False)

Применения фильтра Хампеля

In [3]:
import numpy as np
from sktime.transformations.series.outlier_detection import HampelFilter

In [67]:
result_15_read = pd.read_csv('Данные_валидации/modis_validation_ext_15_distance.csv')
result_15_read

,Time,T,Lat,Lon,Image Time,Image T,MODIS,Day/Night,distance_to_shore_m,T diff
0,2021-05-27 11:02:10+08:00,2.22800,52.465443,106.586891,2021-05-27 11:06:00+08:00,2.35,Modis11_Terra,Day,10263.662946,0.12200
1,2021-05-28 11:37:54+08:00,1.42000,52.052254,105.978180,2021-05-28 11:48:00+08:00,1.69,Modis11_Terra,Day,13368.793886,0.27000
2,2021-06-03 10:58:21+08:00,2.44340,51.606743,104.731491,2021-06-03 11:12:00+08:00,3.05,Modis11_Terra,Day,14584.820130,0.60660
3,2021-06-04 11:49:31+08:00,2.57930,51.626816,105.359306,2021-06-04 11:54:00+08:00,4.11,Modis11_Terra,Day,3211.487032,1.53070
4,2021-06-05 10:50:54+08:00,10.08810,52.292828,106.244370,2021-06-05 11:00:00+08:00,10.57,Modis11_Terra,Day,1620.903932,0.48190
...,...,...,...,...,...,...,...,...,...,...
807,2024-09-21 03:25:00+08:00,10.60505,51.876760,105.205520,2024-09-20 19:24:00+00:00,7.93,Modis21_Aqua,Night,3760.460237,-2.67505
808,2024-09-22 03:58:00+08:00,11.99560,52.100490,106.120810,2024-09-21 20:12:00+00:00,11.61,Modis21_Aqua,Night,6311.388764,-0.38560
809,2024-09-23 05:01:00+08:00,11.86775,52.575820,106.606930,2024-09-22 20:48:00+00:00,10.59,Modis21_Aqua,Night,13472.346742,-1.27775
810,2024-09-25 03:36:00+08:00,12.23610,53.521020,108.145930,2024-09-24 19:48:00+00:00,9.59,Modis21_Aqua,Night,24556.604522,-2.64610


In [68]:
result_15_read[abs(result_15_read["T diff"]) > 4]

,Time,T,Lat,Lon,Image Time,Image T,MODIS,Day/Night,distance_to_shore_m,T diff
7,2021-07-13 11:53:40+08:00,6.4335,55.364616,109.638123,2021-07-13 12:00:00+08:00,10.85,Modis11_Terra,Day,9963.011983,4.4165
37,2022-06-09 11:33:50+08:00,3.8433,55.348057,109.201889,2022-06-09 11:36:00+08:00,10.53,Modis11_Terra,Day,357.809427,6.6867
44,2022-07-30 10:13:28+08:00,13.3911,51.735512,104.520958,2022-07-30 10:24:00+08:00,8.65,Modis11_Terra,Day,6401.024819,-4.7411
46,2022-08-03 11:31:59+08:00,13.0099,54.866493,109.188484,2022-08-03 11:36:00+08:00,6.65,Modis11_Terra,Day,16302.132251,-6.3599
47,2022-08-04 10:37:14+08:00,13.9734,55.648743,109.369965,2022-08-04 10:42:00+08:00,22.13,Modis11_Terra,Day,99.007820,8.1566
...,...,...,...,...,...,...,...,...,...,...
780,2022-10-02 02:48:55+08:00,8.7640,52.002270,105.389015,2022-10-01 19:00:00+00:00,-5.15,Modis21_Aqua,Night,1709.454702,-13.9140
783,2023-07-22 03:18:08+08:00,15.4022,53.453381,108.738663,2023-07-21 19:24:00+00:00,20.41,Modis21_Aqua,Night,10988.388235,5.0078
795,2023-10-23 02:22:45+08:00,8.6722,53.452267,107.574905,2023-10-22 18:36:00+00:00,-2.15,Modis21_Aqua,Night,2039.030057,-10.8222
796,2023-10-25 03:56:05+08:00,7.8000,53.033154,106.924583,2023-10-24 20:00:00+00:00,1.55,Modis21_Aqua,Night,328.129590,-6.2500


In [40]:
result_15_read = result_15_read.sample(frac=1).reset_index(drop=True)
result_15_read["T diff"]

0      6.267950
1      0.585950
2      0.240878
3     -2.901250
4     -7.960900
         ...   
807    3.010100
808    0.155500
809   -5.018500
810    0.556100
811    2.304800
Name: T diff, Length: 812, dtype: float64

In [69]:
transformer = HampelFilter(window_length=70)
result_filter_outliers = transformer.fit_transform(result_15_read["T diff"])
result_filter_outliers[result_filter_outliers.notnull()].count

<bound method Series.count of 0      0.12200
1      0.27000
2      0.60660
3      1.53070
4      0.48190
        ...   
806   -1.74815
807   -2.67505
808   -0.38560
809   -1.27775
810   -2.64610
Name: T diff, Length: 730, dtype: float64>

In [70]:
result_15_read_filtered = result_15_read
result_15_read_filtered["T diff"] = result_filter_outliers
result_15_read_filtered = result_15_read_filtered[result_15_read_filtered["T diff"].notnull()]
result_15_read_filtered

,Time,T,Lat,Lon,Image Time,Image T,MODIS,Day/Night,distance_to_shore_m,T diff
0,2021-05-27 11:02:10+08:00,2.22800,52.465443,106.586891,2021-05-27 11:06:00+08:00,2.35,Modis11_Terra,Day,10263.662946,0.12200
1,2021-05-28 11:37:54+08:00,1.42000,52.052254,105.978180,2021-05-28 11:48:00+08:00,1.69,Modis11_Terra,Day,13368.793886,0.27000
2,2021-06-03 10:58:21+08:00,2.44340,51.606743,104.731491,2021-06-03 11:12:00+08:00,3.05,Modis11_Terra,Day,14584.820130,0.60660
3,2021-06-04 11:49:31+08:00,2.57930,51.626816,105.359306,2021-06-04 11:54:00+08:00,4.11,Modis11_Terra,Day,3211.487032,1.53070
4,2021-06-05 10:50:54+08:00,10.08810,52.292828,106.244370,2021-06-05 11:00:00+08:00,10.57,Modis11_Terra,Day,1620.903932,0.48190
...,...,...,...,...,...,...,...,...,...,...
806,2024-09-20 04:11:00+08:00,13.05815,51.608270,105.100390,2024-09-19 20:12:00+00:00,11.31,Modis21_Aqua,Night,5086.948500,-1.74815
807,2024-09-21 03:25:00+08:00,10.60505,51.876760,105.205520,2024-09-20 19:24:00+00:00,7.93,Modis21_Aqua,Night,3760.460237,-2.67505
808,2024-09-22 03:58:00+08:00,11.99560,52.100490,106.120810,2024-09-21 20:12:00+00:00,11.61,Modis21_Aqua,Night,6311.388764,-0.38560
809,2024-09-23 05:01:00+08:00,11.86775,52.575820,106.606930,2024-09-22 20:48:00+00:00,10.59,Modis21_Aqua,Night,13472.346742,-1.27775


In [71]:
result_15_read_filtered.max()

Time                   2024-10-02 13:38:00+08:00
T                                        21.7039
Lat                                     55.73514
Lon                                    109.90421
Image Time             2024-10-02 11:00:00+08:00
Image T                                    24.93
MODIS                              Modis21_Terra
Day/Night                                  Night
distance_to_shore_m                 29807.153895
T diff                                    4.9275
dtype: object

In [65]:
result_15_read_filtered.to_csv('Данные_валидации/modis_validation_ext_15_distance_filtered4.csv', index=False)

## Landsat

In [ ]:
result_1_read = pd.read_csv('Данные_валидации/landsat_ext_validation1.csv')
result_1_read

,Time,T,Lat,Lon,Image Time,Image T,MODIS,Day/Night
0,2021-05-27 11:02:10+08:00,2.22800,52.465443,106.586891,2021-05-27 11:06:00+08:00,2.35,Modis11_Terra,Day
1,2021-05-28 11:37:54+08:00,1.42000,52.052254,105.978180,2021-05-28 11:48:00+08:00,1.69,Modis11_Terra,Day
2,2021-06-03 10:58:21+08:00,2.44340,51.606743,104.731491,2021-06-03 11:12:00+08:00,3.05,Modis11_Terra,Day
3,2021-06-04 11:49:31+08:00,2.57930,51.626816,105.359306,2021-06-04 11:54:00+08:00,4.11,Modis11_Terra,Day
4,2021-06-05 10:50:54+08:00,10.08810,52.292828,106.244370,2021-06-05 11:00:00+08:00,10.57,Modis11_Terra,Day
...,...,...,...,...,...,...,...,...
807,2024-09-21 03:25:00+08:00,10.60505,51.876760,105.205520,2024-09-20 19:24:00+00:00,7.93,Modis21_Aqua,Night
808,2024-09-22 03:58:00+08:00,11.99560,52.100490,106.120810,2024-09-21 20:12:00+00:00,11.61,Modis21_Aqua,Night
809,2024-09-23 05:01:00+08:00,11.86775,52.575820,106.606930,2024-09-22 20:48:00+00:00,10.59,Modis21_Aqua,Night
810,2024-09-25 03:36:00+08:00,12.23610,53.521020,108.145930,2024-09-24 19:48:00+00:00,9.59,Modis21_Aqua,Night


In [ ]:
point_distance = result_1_read[['Lon','Lat']].apply(lambda x: (ee.Geometry.Point(*x)).distance(baikal_points).getInfo(), axis=1)
result_1_with_distance = result_1_read.assign(distance_to_shore_m=pd.Series(point_distance).values)

In [ ]:
result_1_with_distance.to_csv('Данные_валидации/landsat_ext_validation1_distance.csv', index=False)

Добавление колонки с разностью

In [2]:
result_1_read = pd.read_csv('Данные_валидации/landsat_ext_validation1_distance.csv')
result_1_read

,Time,T,Lat,Lon,Image Time,Image T,Landsat,distance_to_shore_m
0,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:29.542497+00:00,9.714929,L8,10555.231491
1,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:53.420828900+00:00,9.716510,L8,10555.231491
2,2021-07-29 11:39:00+08:00,11.988600,53.443645,108.509331,2021-07-29 03:39:24.750487+00:00,13.214574,L8,5520.731797
3,2021-08-14 11:38:37+08:00,15.308000,53.481445,108.645866,2021-08-14 03:39:31.126727+00:00,16.345016,L8,5401.557395
4,2021-09-22 11:45:11+08:00,9.044000,52.911846,107.182930,2021-09-22 03:45:51.413924900+00:00,8.886449,L8,15331.670909
5,2024-07-28 11:45:00+08:00,19.927840,52.449190,106.141990,2024-07-28 03:45:34.798499900+00:00,20.237132,L8,10132.228022
6,2022-06-12 11:51:29+08:00,3.179000,53.250637,107.276199,2022-06-12 03:51:52.862325+00:00,4.722633,L8,6597.664926
7,2021-08-12 11:51:29+08:00,13.258200,51.912121,105.253105,2021-08-12 03:52:15.999436+00:00,14.475679,L8,1424.743795
8,2022-07-30 11:51:58+08:00,13.221900,51.751106,104.527298,2022-07-30 03:52:32.544500+00:00,12.992808,L8,4892.587084
9,2023-09-19 11:51:53+08:00,15.168700,51.527775,104.239059,2023-09-19 03:52:12.568362+00:00,14.582771,L8,2080.802453


In [3]:
result_1_read["T diff"] = result_1_read["Image T"] - result_1_read["T"]
result_1_read.to_csv('Данные_валидации/landsat_ext_validation1_distance.csv', index=False)

Применения фильтра Хампеля

In [5]:
result_1_read = pd.read_csv('Данные_валидации/landsat_ext_validation1_distance.csv')
result_1_read

,Time,T,Lat,Lon,Image Time,Image T,Landsat,distance_to_shore_m,T diff
0,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:29.542497+00:00,9.714929,L8,10555.231491,2.636729
1,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:53.420828900+00:00,9.716510,L8,10555.231491,2.638310
2,2021-07-29 11:39:00+08:00,11.988600,53.443645,108.509331,2021-07-29 03:39:24.750487+00:00,13.214574,L8,5520.731797,1.225974
3,2021-08-14 11:38:37+08:00,15.308000,53.481445,108.645866,2021-08-14 03:39:31.126727+00:00,16.345016,L8,5401.557395,1.037016
4,2021-09-22 11:45:11+08:00,9.044000,52.911846,107.182930,2021-09-22 03:45:51.413924900+00:00,8.886449,L8,15331.670909,-0.157551
5,2024-07-28 11:45:00+08:00,19.927840,52.449190,106.141990,2024-07-28 03:45:34.798499900+00:00,20.237132,L8,10132.228022,0.309292
6,2022-06-12 11:51:29+08:00,3.179000,53.250637,107.276199,2022-06-12 03:51:52.862325+00:00,4.722633,L8,6597.664926,1.543633
7,2021-08-12 11:51:29+08:00,13.258200,51.912121,105.253105,2021-08-12 03:52:15.999436+00:00,14.475679,L8,1424.743795,1.217479
8,2022-07-30 11:51:58+08:00,13.221900,51.751106,104.527298,2022-07-30 03:52:32.544500+00:00,12.992808,L8,4892.587084,-0.229092
9,2023-09-19 11:51:53+08:00,15.168700,51.527775,104.239059,2023-09-19 03:52:12.568362+00:00,14.582771,L8,2080.802453,-0.585929


In [6]:
transformer = HampelFilter(window_length=7)
result_filter_outliers = transformer.fit_transform(result_1_read["T diff"])
result_filter_outliers

0     2.636729
1     2.638310
2     1.225974
3     1.037016
4    -0.157551
5     0.309292
6     1.543633
7     1.217479
8    -0.229092
9    -0.585929
10   -0.174143
11    0.400046
12   -0.264850
13   -0.435137
14   -0.470053
15   -1.320315
16    2.250982
17   -1.299574
Name: T diff, dtype: float64

In [7]:
result_1_read["T diff"] = result_filter_outliers
result_1_read = result_1_read[result_1_read["T diff"].notnull()]
result_1_read

,Time,T,Lat,Lon,Image Time,Image T,Landsat,distance_to_shore_m,T diff
0,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:29.542497+00:00,9.714929,L8,10555.231491,2.636729
1,2021-07-13 11:38:04+08:00,7.078200,55.332798,109.608627,2021-07-13 03:38:53.420828900+00:00,9.716510,L8,10555.231491,2.638310
2,2021-07-29 11:39:00+08:00,11.988600,53.443645,108.509331,2021-07-29 03:39:24.750487+00:00,13.214574,L8,5520.731797,1.225974
3,2021-08-14 11:38:37+08:00,15.308000,53.481445,108.645866,2021-08-14 03:39:31.126727+00:00,16.345016,L8,5401.557395,1.037016
4,2021-09-22 11:45:11+08:00,9.044000,52.911846,107.182930,2021-09-22 03:45:51.413924900+00:00,8.886449,L8,15331.670909,-0.157551
5,2024-07-28 11:45:00+08:00,19.927840,52.449190,106.141990,2024-07-28 03:45:34.798499900+00:00,20.237132,L8,10132.228022,0.309292
6,2022-06-12 11:51:29+08:00,3.179000,53.250637,107.276199,2022-06-12 03:51:52.862325+00:00,4.722633,L8,6597.664926,1.543633
7,2021-08-12 11:51:29+08:00,13.258200,51.912121,105.253105,2021-08-12 03:52:15.999436+00:00,14.475679,L8,1424.743795,1.217479
8,2022-07-30 11:51:58+08:00,13.221900,51.751106,104.527298,2022-07-30 03:52:32.544500+00:00,12.992808,L8,4892.587084,-0.229092
9,2023-09-19 11:51:53+08:00,15.168700,51.527775,104.239059,2023-09-19 03:52:12.568362+00:00,14.582771,L8,2080.802453,-0.585929


In [8]:
result_1_read.to_csv('Данные_валидации/landsat_ext_validation1_distance_filtered.csv', index=False)